In [1]:
pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
pip install dash dash-bootstrap-components pandas python-jobspy

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
pip install -U python-jobspy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 962.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: NUMPY
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [ ]:
import pandas as pd
from dash import Dash, html, dcc, dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from jobspy import scrape_jobs

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the app layout
app.layout = dbc.Container(
    [
        html.H1("Job Search Aggregator"),
        html.P(
            "Enter up to 5 search terms below to find the latest job listings. "
            "Leave fields blank if not needed. Results will be combined and duplicates removed."
        ),
        # Input fields for search terms
        dbc.Form(
            [
                dbc.Row(
                    [
                        dbc.Label(f"Search Term {i}", width=2),
                        dbc.Col(
                            dbc.Input(
                                id=f"search_term_{i}",
                                type="text",
                                placeholder=f"Enter search term {i}",
                            ),
                            width=10,
                        ),
                    ],
                    className="mb-3",
                )
                for i in range(1, 6)
            ]
        ),
        # Input field for location
        dbc.Row(
            [
                dbc.Label("Location", width=2),
                dbc.Col(
                    dbc.Input(
                        id="location",
                        type="text",
                        placeholder="Enter location (e.g., Iowa City, IA)",
                        value="Iowa City, IA",  # Default value
                    ),
                    width=10,
                ),
            ],
            className="mb-3",
        ),
        # Search button
        dbc.Button("Search Jobs", id="search_button", color="primary", className="mb-3"),
        # Container for displaying messages
        html.Div(id="message", className="mb-3"),
        # DataTable to display job listings
        dash_table.DataTable(
            id="job_table",
            columns=[
                {"name": "Title", "id": "title"},
                {"name": "Company", "id": "company"},
                {"name": "Location", "id": "location"},
                {"name": "Date Posted", "id": "date_posted"},
                {"name": "Job URL", "id": "job_url"},
            ],
            data=[],
            style_table={"overflowX": "auto"},
            style_cell={"textAlign": "left"},
            style_header={"fontWeight": "bold"},
        ),
    ],
    fluid=True,
)

# Define callback to update job listings
@app.callback(
    Output("job_table", "data"),
    Output("message", "children"),
    Input("search_button", "n_clicks"),
    [State(f"search_term_{i}", "value") for i in range(1, 6)],
    State("location", "value"),
)
def update_job_table(n_clicks, *search_terms, location):
    if n_clicks is None:
        return [], ""

    # Filter out empty search terms
    search_terms = [term for term in search_terms if term]

    if not search_terms:
        search_terms = ["Business Analytics, Finance"]  # Default term

    all_jobs = pd.DataFrame()

    for term in search_terms:
        jobs = scrape_jobs(
            site_name=["indeed", "linkedin", "zip_recruiter", "glassdoor"],
            search_term=term,
            google_search_term="masters accounting in iowa city",
            location=location,
            results_wanted=20,
            hours_old=240,
            country_indeed='USA',
        )
        all_jobs = pd.concat([all_jobs, jobs], ignore_index=True)

    # Remove duplicates
    all_jobs = all_jobs.drop_duplicates()

    if all_jobs.empty:
        return [], "No jobs found for the given search terms and location."

    # Convert DataFrame to dictionary for DataTable
    data = all_jobs.to_dict("records")
    message = f"Found {len(all_jobs)} unique jobs across all search terms."

    return data, message

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)



<IPython.core.display.Javascript object>